## This program builds the VGG16 convolutional neural network model for image classification. Once built the model is stored as car_v16.h5 file in the same directory.

In [1]:
# Importing the Keras libraries and packages
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras import applications
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import matplotlib.pyplot as plt
from keras import optimizers 
#remove warnings 
import warnings
warnings.filterwarnings('ignore')
#import config file
import import_ipynb
import Config as c

Using TensorFlow backend.


importing Jupyter notebook from Config.ipynb


In [2]:
# Pre-Trained CNN Model using imagenet dataset for pre-trained weights
base_model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (c.img_width, c.img_height, c.channels))
# Freeze layer that do not need to be trained
for layer in base_model.layers[:10]:
    layer.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Top Model Block
x = base_model.output
x = Flatten()(x)
predictions = Dense(c.num_classes, activation="softmax")(x)

In [4]:
# add your top layer block to your base model
classifier = Model(base_model.input, predictions)
#print(classifier.summary())

In [5]:
# compiling the CNN
classifier.compile(optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [6]:
# Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
# To save augmentations un-comment save lines and add to your flow parameters.
img_width, img_height = c.img_width, c.img_height
train_data_dir = c.train_data_dir
validation_data_dir = c.validation_data_dir
batch_size = c.batch_size
epochs = c.epoch
num_classes = c.num_classes

# Initiate the train and test generators with data Augumentation


train_datagen = ImageDataGenerator(
rescale = c.train_rescale,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = c.train_zoom_range,
width_shift_range = c.train_width_shift_range,
height_shift_range=c.train_height_shift_range,
rotation_range=c.train_rotation_range)

test_datagen = ImageDataGenerator(
rescale = c.test_rescale,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = c.test_zoom_range,
width_shift_range = c.test_width_shift_range,
height_shift_range=c.test_height_shift_range,
rotation_range=c.test_rotation_range)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
class_mode = "categorical")

Found 398 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [7]:
# Define when to stop training the model and then store the best model as car1.h5
# This will be passed while fitting the model as callbacks methods.
checkpoint = ModelCheckpoint(c.model_file, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto') 

In [8]:
# Find the count for train and test images
import os
train_samples = 0
validation_samples = 0

cwd = os.getcwd()
folder = c.train_data_dir
for sub_folder in os.listdir(folder):
    path, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    train_samples += len(files)


folder = c.validation_data_dir
for sub_folder in os.listdir(folder):
    path, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    validation_samples += len(files)

print(train_samples)
print(validation_samples)

398
164


In [9]:
# Fit the model to the data
history_object = classifier.fit_generator(
train_generator,
steps_per_epoch = train_samples,
epochs = c.epoch,
callbacks = [checkpoint, early],
validation_data = validation_generator,
validation_steps = validation_samples
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
398/398 [==============================] - 741s 2s/step - loss: 0.0928 - acc: 0.9632 - val_loss: 0.1048 - val_acc: 0.9488

Epoch 00001: val_acc improved from -inf to 0.94880, saving model to car_v16.h5
Epoch 2/10
398/398 [==============================] - 766s 2s/step - loss: 0.0131 - acc: 0.9969 - val_loss: 0.0966 - val_acc: 0.9590

Epoch 00002: val_acc improved from 0.94880 to 0.95904, saving model to car_v16.h5
Epoch 3/10
398/398 [==============================] - 973s 2s/step - loss: 0.0054 - acc: 0.9993 - val_loss: 0.0937 - val_acc: 0.9619

Epoch 00003: val_acc improved from 0.95904 to 0.96192, saving model to car_v16.h5
Epoch 4/10
398/398 [==============================] - 695s 2s/step - loss: 0.0031 - acc: 0.9997 - val_loss: 0.0918 - val_acc: 0.9653

Epoch 00004: val_acc improved from 0.96192 to 0.96527, saving model to car_v16.h5
Epoch 5/10
398/398 [==============================] - 737s 2s/step - loss: 0.0023 - acc: 0.

In [ ]:
# # %matplotlib inline
# print(history_object.history.keys())
# plt.plot(history_object.history['acc'])
# plt.plot(history_object.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('acc')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# plt.plot(history_object.history['loss'])
# plt.plot(history_object.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()